# Создаём и редактируем дерево категорий

In [1]:
from os import listdir
import jsonlines
import itertools
from tqdm import tqdm
from tqdm import trange
import re
import nltk
import networkx as nx
import re
import matplotlib.pyplot as plt
from collections import Counter
from mediawiki import MediaWiki
from mediawiki import PageError
from mediawiki import MediaWikiCategoryTreeError
from os import sys
from time import sleep

wiki = MediaWiki(url='http://ru.wikipedia.org/w/api.php')
tknzr = nltk.TweetTokenizer()
#nltk.download()

## Достаём связи между категориями
(node_2_id) $\rightarrow$ (node_1_id)

In [2]:
links = [link for link in tqdm(jsonlines.open('links/link_directed.txt', 'r'))]

719813it [00:05, 130669.33it/s]


In [3]:
categories_id_dict = {}
for link in tqdm(links):
    categories_id_dict[str(link['node_1_id'])] = re.sub(r'^Категория:', '', link['node_1_title'])
    categories_id_dict[str(link['node_2_id'])] = re.sub(r'^Категория:', '', link['node_2_title'])

100%|██████████████████████████████████████████████████████████████████████| 719813/719813 [00:04<00:00, 176647.99it/s]


In [4]:
categories_forest = nx.DiGraph()
categories_forest.add_edges_from([(str(link['node_2_id']), str(link['node_1_id'])) for link in links])

print("Number of nodes:", categories_forest.number_of_nodes())
print("Number of edges:", categories_forest.number_of_edges())
print("Is it a forest?", nx.is_forest(categories_forest))
print("Connected components:", nx.number_weakly_connected_components(categories_forest))

Number of nodes: 344614
Number of edges: 719389
Is it a forest? False
Connected components: 58


In [5]:
components = list(nx.weakly_connected_component_subgraphs(categories_forest))

In [6]:
for c in components:
    roots = []
    for node in list(c.nodes()):
        if len(list(c.predecessors(node))) == 0:
            roots.append(node)
            
    print([categories_id_dict[root] for root in roots])
            

['Статьи проекта Екатеринбург по важности', 'Статьи проекта Театр III уровня', 'Русские коллаборационисты с нацистской Германией', 'Статьи проекта Хоррор II уровня', 'Статьи проекта Удмуртия низкой важности', 'Статьи проекта Подводные лодки по уровню', 'Статьи проекта Эволюционная биология по важности', 'Википедия:Страницы на КПМ по типам', 'Незавершённые статьи о подвижном составе', 'Статьи проекта Современное искусство по уровню', 'Статьи проекта Немецкий футбол по уровню', 'Статьи проекта Грузия по важности', 'Статьи проекта Самара по важности', 'Статьи проекта Театр средней важности', 'Статьи проекта Харьков по уровню', 'Статьи проекта Ленин по важности', 'Статьи проекта Ракетное оружие по важности', 'Хорошие статьи по проектам', 'Незавершённые статьи о сооружениях', 'Статьи проекта Английский футбол по важности', 'Статьи проекта Югра по важности', 'Википедия:Страницы на КУЛ по странам', 'Статьи проекта Крейсера по уровню', 'Статьи проекта Шашки по уровню', 'Статьи проекта Анатомия


['Статьи проекта Россия II уровня']
['Статьи проекта Наука по качеству и важности']
['Статьи проекта История России неизвестного уровня']
['Статьи проекта Русский рок I уровня']
['Статьи проекта Дания II уровня']
['Статьи проекта Монголия низкой важности']
['Статьи проекта История России I уровня']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Хорошие статьи проекта История России']
['Избранные статьи проекта История России']
['Статьи проекта Военная история II уровня']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Статьи проекта Антарктика II уровня']
['Статьи проекта Антарктика III уровня']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [7]:
for i in range(len(components)):
    print(i)
    print("Number of nodes:", components[i].number_of_nodes())
    print("Number of edges:", components[i].number_of_edges())

0
Number of nodes: 344531
Number of edges: 719318
1
Number of nodes: 6
Number of edges: 5
2
Number of nodes: 2
Number of edges: 1
3
Number of nodes: 4
Number of edges: 3
4
Number of nodes: 4
Number of edges: 3
5
Number of nodes: 2
Number of edges: 1
6
Number of nodes: 2
Number of edges: 1
7
Number of nodes: 4
Number of edges: 3
8
Number of nodes: 1
Number of edges: 1
9
Number of nodes: 1
Number of edges: 1
10
Number of nodes: 1
Number of edges: 1
11
Number of nodes: 1
Number of edges: 1
12
Number of nodes: 1
Number of edges: 1
13
Number of nodes: 1
Number of edges: 1
14
Number of nodes: 1
Number of edges: 1
15
Number of nodes: 1
Number of edges: 1
16
Number of nodes: 1
Number of edges: 1
17
Number of nodes: 1
Number of edges: 1
18
Number of nodes: 1
Number of edges: 1
19
Number of nodes: 1
Number of edges: 1
20
Number of nodes: 1
Number of edges: 1
21
Number of nodes: 1
Number of edges: 1
22
Number of nodes: 2
Number of edges: 2
23
Number of nodes: 1
Number of edges: 1
24
Number of nod

### Поняли, что нас интересует только первая компонента, где есть Категория:Всё

In [18]:
root = 0
def srch_by_ttl(ttl):
    for id, title in categories_id_dict.items():
        if title == ttl:
            print(id)
            return id

root = srch_by_ttl('Всё')

def del_by_ttls(ttls):
    for ttl in ttls:
        components[0].remove_node(srch_by_ttl(ttl))
        
   
#del_by_ttls(['Википедия:Служебные', 'Скрытые категории'])
'''
for id, title in categories_id_dict.items():
    if title == 'Википедия:Служебные':
        print(id)
        components[0].remove_node(id)
        break'''
        
srch_by_ttl('Аскеты')

16350


In [18]:
class BuldTaxonomy():
    def __init__(self, graph, root):
        self.original_graph = graph
        self.output_graph = nx.DiGraph()
        self.stack = [root]
        
    '''
    def build_graph_recursive(self, node):
        successors = list(self.original_graph.successors(node))
        if len(successors) == 0:
            return
        else:
            for pred in successors:
                self.output_graph.add_edge(node, succ)
                self.buld_graph_recursive(succ)
    '''
    
    def build_graph(self):
        while self.stack:
            node = self.stack.pop()
            successors = list(self.original_graph.successors(node))
            for succ in successors:
                if (node, succ) not in self.output_graph.edges():
                    self.output_graph.add_edge(node, succ)
                    self.stack.append(succ)

In [20]:
taxonomy = BuldTaxonomy(components[0], srch_by_ttl('Всё'))
taxonomy.build_graph()
cat_graph = taxonomy.output_graph
cat_tree = nx.bfs_tree(components[0], root)


print("Number of nodes:", cat_graph.number_of_nodes())
print("Number of edges:", cat_graph.number_of_edges())

print("Number of nodes:", cat_tree.number_of_nodes())
print("Number of edges:", cat_tree.number_of_edges())

nx.write_gpickle(cat_graph, 'sources/graph_gpickle.gpickle')
nx.write_gpickle(cat_tree, 'sources/tree_gpickle.gpickle')

16350
Number of nodes: 316892
Number of edges: 667517
Number of nodes: 316892
Number of edges: 316891


In [33]:
lengths = nx.single_source_shortest_path_length(cat_graph, srch_by_ttl('Всё'))
Counter(lengths.values())

16350


Counter({0: 1,
         1: 2,
         2: 10,
         3: 223,
         4: 3119,
         5: 13939,
         6: 60192,
         7: 91053,
         8: 89664,
         9: 43536,
         10: 11060,
         11: 2634,
         12: 888,
         13: 300,
         14: 206,
         15: 37,
         16: 24,
         17: 4})

In [11]:
%matplotlib notebook

# Википедия:Категории, объединённые по внутренним признакам
nodes = [srch_by_ttl('Статьи')]
lil_graphy = nx.DiGraph()
for i in trange(2):
    new_nodes = []
    for node in nodes:
        successors = list(cat_graph.successors(node))
        new_nodes.extend(successors)
        for succ in successors:
            lil_graphy.add_edge(categories_id_dict[node], categories_id_dict[succ])
    nodes = new_nodes
    
print('Nodes:', lil_graphy.number_of_nodes())
pos = nx.circular_layout(lil_graphy)
nx.draw_networkx_nodes(lil_graphy,pos,node_size=1600, 
                           alpha=0.3, node_color='red')
nx.draw_networkx_edges(lil_graphy,pos,width=1,
                           alpha=0.3,edge_color='black')
nx.draw_networkx_labels(lil_graphy, pos,font_size=10,
                            font_family='sans-serif')
nx.draw(lil_graphy, pos)
plt.show()

5861034


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]


NameError: name 'cat_graph' is not defined